In [162]:
import pandas as pd
from pandas import ExcelWriter 
pd.__version__

'1.0.0'

In [344]:
class JTLReader:
    def __init__(self,jtlPath):
        self.jtlPath=jtlPath
        self.jtlColumns = ['timeStamp', 'elapsed','label','bytes','allThreads','responseCode']
        %time self.df_Jtl=pd.read_csv(self.jtlPath,usecols=self.jtlColumns,dtype={'label':'category'})#,nrows=100     
        self.shape=self.df_Jtl.shape
        self.rows=self.shape[0]
        self.rows         
        
    def converTimeStamp(self):
        self.df_Jtl['timeStamp'] = pd.to_datetime(self.df_Jtl['timeStamp'],unit='ms')
        self.df_Jtl['timeStamp'] = self.df_Jtl['timeStamp'].astype('datetime64[s]')#reduce precision pandas timestamp dataframe
        self.df_Jtl['timeStamp']=self.df_Jtl['timeStamp'].dt.tz_localize('utc').dt.tz_convert('Singapore')
        self.df_Jtl['timeStamp'] = self.df_Jtl['timeStamp'].astype(str).str[:-6]
        self.df_Jtl['timeStamp'] = pd.to_datetime(self.df_Jtl['timeStamp'])
        self.df_Jtl['timeStamp'] = self.df_Jtl['timeStamp'].astype('datetime64[s]') 
        
    def ms_to_sec(self):
         self.df_Jtl['elapsed']=(self.df_Jtl['elapsed']/1000)  
        
    def overAllResponseTimeSummary(self):
        self.df_overAllResponseTimeSummary=self.df_Jtl[['timeStamp','elapsed']]
        return self.df_overAllResponseTimeSummary
    
    def responseTimeSummary(self):
        self.df_responseTimeSummary=self.df_Jtl[['label','elapsed']]
        self.df_txnGroup=self.df_responseTimeSummary.groupby(['label'])
        self.df_txnGroup=self.df_txnGroup.elapsed.describe(percentiles=[.90])#self.df_txnGroup.describe(percentiles=[.90])
        self.df_txnGroup.columns=self.df_txnGroup.columns.get_level_values(0)#self.df_txnGroup.columns=self.df_txnGroup.columns.get_level_values(1)
        self.df_txnGroup = self.df_txnGroup[['min','mean','90%','max','count']]
        #self.df_txnGroup=self.df_txnErrorGroup.reset_index() """To be enable if you run describe () on data frame. Now we are running on specific column"""
        return self.df_txnGroup
    
    def errorSummary(self):
        self.df_errorSummary=self.df_Jtl[['label','responseCode']]
        self.df_txnErrorGroup=self.df_errorSummary.groupby(['label'])
        self.df_txnErrorGroup=self.df_txnErrorGroup.responseCode.describe()
        return self.df_txnErrorGroup
    
        
    def calculateErrorPercentage(self,responseTimeSummary,errorSummary):
        #Method 1
        #self.txn_grp=self.df_Jtl.groupby(['label'])
        #return self.txn_grp.get_group('BP01_TX05_Submit')
        
        #Method 2
        #filt=self.df_Jtl['label']=='BP01_TX05_Submit'
        #return self.df_Jtl.loc[filt]['responseCode'].value_counts()
        
        #Method 3
        #self.txn_grp=self.df_Jtl.groupby(['label'])
        #return self.txn_grp['responseCode'].value_counts(normalize=True)
        
        self.df_responseTimeSummary=responseTimeSummary
        self.df_errorSummary=errorSummary
        self.response_error=pd.merge(self.df_responseTimeSummary,self.df_errorSummary)
        return self.response_error
        
        
        
        return self.txn_grp
    
    def listOfErrors(self):
        return self.df_Jtl['responseCode'].value_counts()


       
        
    
        
obj1=JTLReader("D:\\Pandas\\data\\thalas_result.jtl")
obj1.converTimeStamp()
obj1.ms_to_sec() 



Wall time: 20.4 ms


In [287]:
obj1.listOfErrors()

200                                                                      6074
Non HTTP response code: org.apache.http.conn.HttpHostConnectException      12
Name: responseCode, dtype: int64

In [329]:
#overAllResponseTimeSummary
obj1.overAllResponseTimeSummary()


,timeStamp,elapsed
0,2020-03-27 11:43:48,0.611
1,2020-03-27 11:43:48,0.611
2,2020-03-27 11:43:48,0.588
3,2020-03-27 11:43:48,0.588
4,2020-03-27 11:43:48,1.508
...,...,...
6081,2020-03-27 12:12:19,0.804
6082,2020-03-27 12:12:20,0.947
6083,2020-03-27 12:12:20,0.947
6084,2020-03-27 12:12:20,0.974


In [347]:
#responseTimeSummary
responseTimeSummary=obj1.responseTimeSummary()
responseTimeSummary.index


CategoricalIndex(['/-1', '/login.php-4', '/mercurypurchase.php-11',
                  '/mercurypurchase2.php-16', '/mercuryreservation2.php-8',
                  '/mercurysignoff.php-23', 'BP01_TX01_Launch URL',
                  'BP01_TX02_Login', 'BP01_TX03_Continue1',
                  'BP01_TX04_Continue2', 'BP01_TX05_Logoff',
                  'BP01_TX05_Submit', 'DeleteUserName', 'GetUserName'],
                 categories=['/-1', '/login.php-4', '/mercurypurchase.php-11', '/mercurypurchase2.php-16', '/mercuryreservation2.php-8', '/mercurysignoff.php-23', 'BP01_TX01_Launch URL', 'BP01_TX02_Login', ...], ordered=False, name='label', dtype='category')

In [199]:
#write response summary to excel with txn name,min,avg,max,90% & max
responseTimeSummary=obj1.responseTimeSummary()
writer = ExcelWriter('PythonExport.xlsx')
responseTimeSummary.to_excel(writer,sheet_name='ResponseSummary')
writer.save()

In [338]:
errorSummary=obj1.errorSummary()
errorSummary

,count,unique,top,freq
label,,,,
/-1,508,1,200,508
/login.php-4,508,1,200,508
/mercurypurchase.php-11,506,1,200,506
/mercurypurchase2.php-16,506,2,200,503
/mercuryreservation2.php-8,507,2,200,506
/mercurysignoff.php-23,504,2,200,502
BP01_TX01_Launch URL,508,1,200,508
BP01_TX02_Login,508,1,200,508
BP01_TX03_Continue1,507,2,200,506


In [346]:
ser=obj1.calculateErrorPercentage(responseTimeSummary,errorSummary)
ser

,min,mean,90%,max,count,unique,top,freq
0,0.475,0.661967,0.6890,1.461,508,1,200,508
1,0.475,0.661967,0.6890,1.461,508,1,200,508
2,0.475,0.661967,0.6890,1.461,508,1,200,508
3,0.475,0.661967,0.6890,1.461,508,1,200,508
4,1.445,1.532559,1.5680,2.186,508,1,200,508
5,1.445,1.532559,1.5680,2.186,508,1,200,508
6,1.445,1.532559,1.5680,2.186,508,1,200,508
7,1.445,1.532559,1.5680,2.186,508,1,200,508
8,0.475,0.661967,0.6890,1.461,508,1,200,508
9,0.475,0.661967,0.6890,1.461,508,1,200,508


In [306]:
df=ser.unstack()
df

responseCode,200,Non HTTP response code: org.apache.http.conn.HttpHostConnectException
label,,
/-1,1.000000,NaN
/login.php-4,1.000000,NaN
/mercurypurchase.php-11,1.000000,NaN
/mercurypurchase2.php-16,0.994071,0.005929
/mercuryreservation2.php-8,0.998028,0.001972
/mercurysignoff.php-23,0.996032,0.003968
BP01_TX01_Launch URL,1.000000,NaN
BP01_TX02_Login,1.000000,NaN
BP01_TX03_Continue1,0.998028,0.001972


In [307]:
df.loc[:,'Non HTTP response code: org.apache.http.conn.HttpHostConnectException']

label
/-1                                NaN
/login.php-4                       NaN
/mercurypurchase.php-11            NaN
/mercurypurchase2.php-16      0.005929
/mercuryreservation2.php-8    0.001972
/mercurysignoff.php-23        0.003968
BP01_TX01_Launch URL               NaN
BP01_TX02_Login                    NaN
BP01_TX03_Continue1           0.001972
BP01_TX04_Continue2                NaN
BP01_TX05_Logoff              0.003968
BP01_TX05_Submit              0.005929
DeleteUserName                     NaN
GetUserName                        NaN
Name: Non HTTP response code: org.apache.http.conn.HttpHostConnectException, dtype: float64